# Squad dataset

In [1]:
from transformers import pipeline, set_seed, AutoModelForCausalLM, AutoTokenizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import os
import string
import time
import random
import json
import torch

set_seed(42)

def completion_judgement(incomplete, next_token):
    tokens = word_tokenize(incomplete)
    if len(tokens)>=100 and next_token == '.':
        return True
    else:
        return False
    
def completion_judgement_improved(incomplete):
    incomplete_tokens = word_tokenize(incomplete)
    if len(incomplete_tokens) >= 150:
        return True
    else:
        return False

def select_model_random():
#     model_list = ['gpt2-xl', 'facebook/opt-2.7b', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b']
    model_list = ['gpt2-xl', 'facebook/opt-2.7b', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B']
    ran = random.randint(0,3)
    ran_model_name = model_list[ran]
    return ran_model_name

# generate_next_token('facebook/opt-2.7b', 'The man worked as a')

/home/fanhuan/anaconda3/envs/detect/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# max_generation_length = 200, add lenght control, adding more parameter controls

def get_model_fullname(model_name):
    return model_fullnames[model_name] if model_name in model_fullnames else model_name

def from_pretrained(cls, model_name, kwargs, cache_dir):
    # use local model if it exists
    local_path = os.path.join(cache_dir, 'local.' + model_name.replace("/", "_"))
    if os.path.exists(local_path):
        return cls.from_pretrained(local_path, **kwargs)
    return cls.from_pretrained(model_name, **kwargs, cache_dir=cache_dir)


def load_tokenizer(model_name, for_dataset, cache_dir):
    model_fullname = get_model_fullname(model_name)
    optional_tok_kwargs = {}
    if "facebook/opt-" in model_fullname:
        print("Using non-fast tokenizer for OPT")
        optional_tok_kwargs['fast'] = False
    if for_dataset in ['pubmed']:
        optional_tok_kwargs['padding_side'] = 'left'
    else:
        optional_tok_kwargs['padding_side'] = 'right'
    base_tokenizer = from_pretrained(AutoTokenizer, model_fullname, optional_tok_kwargs, cache_dir=cache_dir)
    if base_tokenizer.pad_token_id is None:
        base_tokenizer.pad_token_id = base_tokenizer.eos_token_id
        if '13b' in model_fullname:
            base_tokenizer.pad_token_id = 0
    return base_tokenizer

def generate_next_token_improved(model_name, dataset, cache_dir, prompt):
    generator = pipeline('text-generation', model=model_name, model_kwargs={"cache_dir": "/home/fanhuan/cache"})
    min_length = 50
    base_tokenizer = load_tokenizer(model_name, dataset, cache_dir)
    result = generator(prompt, max_length=200, num_return_sequences=1, min_length=min_length, do_sample=True,
                                                   pad_token_id=base_tokenizer.eos_token_id,
                                                   eos_token_id=base_tokenizer.eos_token_id)
    return result

def generate_next_token_improved_GPU(model_name, dataset, cache_dir, prompt, device_num, max_length):
    generator = pipeline('text-generation', model=model_name, model_kwargs={"cache_dir": "/home/fanhuan/cache"}, device=device_num)
    min_length = 30
    base_tokenizer = load_tokenizer(model_name, dataset, cache_dir)
    result = generator(prompt, max_length=max_length, num_return_sequences=1, min_length=min_length, do_sample=True,
                                                   pad_token_id=base_tokenizer.eos_token_id,
                                                   eos_token_id=base_tokenizer.eos_token_id)
    return result

    
def generate_next_token_improved_GPU_one(model_name, dataset, cache_dir, prompt, device_num):
    generator = pipeline('text-generation', model=model_name, model_kwargs={"cache_dir": "/home/fanhuan/cache"}, device=device_num)
    min_length = 1
    base_tokenizer = load_tokenizer(model_name, dataset, cache_dir)
    result = generator(prompt, num_return_sequences=1, min_length=min_length, do_sample=True,
                                                   pad_token_id=base_tokenizer.eos_token_id,
                                                   eos_token_id=base_tokenizer.eos_token_id, max_new_tokens=1)
    return result

In [3]:
# read data file
import pandas as pd
import numpy as np
df = pd.read_json('datasets/fast-detectgpt/squad_gpt2-xl.raw_data.json', orient='records')
df

,original,sampled
0,"In 1954, major Serbian and Croatian writers, l...","In 1954, major Serbian and Croatian writers, l..."
1,The Bush administration then turned its attent...,The Bush administration then turned its attent...
2,An adolescent's environment plays a huge role ...,An adolescent's environment plays a huge role ...
3,"Compared to other team sports, the National Fo...","Compared to other team sports, the National Fo..."
4,"In practice, not all rule violations will resu...","In practice, not all rule violations will resu..."
...,...,...
295,The success of its football team made Notre Da...,The success of its football team made Notre Da...
296,Switzerland lies between latitudes 45° and 48°...,Switzerland lies between latitudes 45° and 48°...
297,Boston has a continental climate with some mar...,Boston has a continental climate with some mar...
298,"In December, Beyoncé along with a variety of o...","In December, Beyoncé along with a variety of o..."


# Speed Up: no inherit, direct using model.generate()

# One-token generation ensemble

In [4]:
model_list = ['gpt2-xl', 'facebook/opt-2.7b', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B']
model_name_1 = model_list[0]
model_name_2 = model_list[1]
model_name_3 = model_list[2]
model_name_4 = model_list[3]

# device_num = 4
cuda_name = 'cuda:5'

# load models, tokenizer and basic EOS and PAD token_id in the GPU first

tokenizer_1 = AutoTokenizer.from_pretrained(model_name_1, cache_dir = '/home/fanhuan/cache')
model_1 = AutoModelForCausalLM.from_pretrained(model_name_1, cache_dir = '/home/fanhuan/cache')
tokenizer_1.pad_token_id = tokenizer_1.eos_token_id

tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2, cache_dir = '/home/fanhuan/cache')
model_2 = AutoModelForCausalLM.from_pretrained(model_name_2, cache_dir = '/home/fanhuan/cache')
tokenizer_2.pad_token_id = tokenizer_2.eos_token_id

tokenizer_3 = AutoTokenizer.from_pretrained(model_name_3, cache_dir = '/home/fanhuan/cache')
model_3 = AutoModelForCausalLM.from_pretrained(model_name_3, cache_dir = '/home/fanhuan/cache')
tokenizer_3.pad_token_id = tokenizer_3.eos_token_id

tokenizer_4 = AutoTokenizer.from_pretrained(model_name_4, cache_dir = '/home/fanhuan/cache')
model_4 = AutoModelForCausalLM.from_pretrained(model_name_4, cache_dir = '/home/fanhuan/cache')
tokenizer_4.pad_token_id = tokenizer_4.eos_token_id

model_1.to(cuda_name)
model_2.to(cuda_name)
model_3.to(cuda_name)
model_4.to(cuda_name)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
  

In [5]:
model_info_list = []
model_info_list.append([model_name_1, tokenizer_1, model_1])
model_info_list.append([model_name_2, tokenizer_2, model_2])
model_info_list.append([model_name_3, tokenizer_3, model_3])
model_info_list.append([model_name_4, tokenizer_4, model_4])

In [6]:
type(model_info_list[0][1])

transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast

In [8]:
def select_model_random_loaded(model_info_list):
        
    model_info = random.choice(model_info_list)
    model_name = model_info[0]
    tokenizer = model_info[1]
    model = model_info[2]
    
    return model_name, tokenizer, model

def completion_judgement(incomplete, next_token):
    tokens = word_tokenize(incomplete)
    if (len(tokens)>=100 and next_token == '.') or (len(tokens)>=150):
        return True
    else:
        return False

In [13]:
%%capture cap

start_it = 0
end_it = 100

max_new_tokens = 1

import pandas as pd
import numpy as np
df = df[:100]
df.to_csv('datasets/suqad-one-token.csv', index=False)

for i in tqdm(range(start_it, end_it)):
    original = df.loc[i,'original']
    token_list = word_tokenize(original) 
    tokens = token_list[:30]
    incomplete = " ".join(tokens)
    flag_completion = False
    
    prompt = f'{incomplete}'
    
    T1 = time.time()

    while(flag_completion is False):
        
        T3 = time.time()
        
        model_name, tokenizer, model = select_model_random_loaded(model_info_list)

        inputs = tokenizer([prompt], return_tensors="pt").to(cuda_name)
        
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, num_return_sequences=1, min_length=1, do_sample=True,
                                                   pad_token_id=tokenizer.eos_token_id,
                                                   eos_token_id=tokenizer.eos_token_id,
                                                   return_dict_in_generate=True, output_scores=True)
    
        input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
        generated_tokens = outputs.sequences[:, input_length:]
        temp_output = tokenizer.decode(generated_tokens[0])
    
        cut_token_number = 0 
        temp_token_list = word_tokenize(temp_output) 
        temp_tokens = temp_token_list[cut_token_number:]
        cut_answer = " ".join(temp_tokens)
        
        flag_completion = completion_judgement(prompt, cut_answer)
        
        if cut_answer in list(string.punctuation):
            prompt = prompt + cut_answer
        else: 
            prompt = " ".join([prompt, cut_answer])

        T4 = time.time()
        time_span = np.round((T4 - T3),2)

    print('The completed version is:', prompt)
    df.loc[i, 'one-token'] = prompt

    T2 = time.time()
    time_span = np.round((T2 - T1),2)
    df.loc[i, 'time-4-one-token'] = time_span
    
    print('---------------------------------------------')
    print('The running time is', (time_span), ' seconds.')
    print('---------------------------------------------')

    df.to_csv('datasets/squad-4-one-token-fast-gpu.csv', index=False)

with open('output/squad-4-one-token-fast-gpu.txt', 'w') as file:
    file.write(cap.stdout)
    
torch.cuda.empty_cache()

  1%|█                                                                                                          | 1/100 [00:09<15:54,  9.64s/it]

The completed version is: In 1954 , major Serbian and Croatian writers , linguists and literary critics , backed by Matica srpska and Matica hrvatska signed the Novi Sad Agreement , which in its main points agreed that Serbia would not declare Yugoslavia 's future political status but would agree to be represented by an agreed representative on Yugoslav issues, that should be agreed between the two countries and that the M atic a hr v ats ka would act as a neutral medi ator to this process. The signing took place at the Nov i z a i a House in Vienna, and several of the country 's authors, like Br ank ic u sv a j is and Slav ko sk a, were invited.
---------------------------------------------
The running time is 9.62  seconds.
---------------------------------------------


  2%|██▏                                                                                                        | 2/100 [00:15<12:28,  7.64s/it]

The completed version is: The Bush administration then turned its attention to Iraq , and argued the need to remove Saddam Hussein from power in Iraq had become urgent . Among the stated reasons was the collapse of the Soviet Union, which they said made Saddam `` a greater threat to the world than we had originally thought.   For the administration, there was no war in Iraq without a pre med itated war with Iraq.    We need to remove Saddam.  We had to justify to the public and to Congress that we are not going to sit id ly by while Iraq is developing ... weapons of mass destruction . ''  He said that the administration did not want to create a power vacuum.
---------------------------------------------
The running time is 6.22  seconds.
---------------------------------------------


  3%|███▏                                                                                                       | 3/100 [00:22<11:38,  7.20s/it]

...

The completed version is: Several explanations have been offered for Yale ’ s representation in national elections since the end of the Vietnam War . Various sources note the spirit of campus activism that took over the country during this era. According to professor Paul Gott fried enberg �` It could be that the Yale students felt that if they stayed silent, we weren � ° s not going to support you. In order to counter this idea, two students formed the Y al ies for America. The other source is that Yale was selected by the Republican Party.
---------------------------------------------
The running time is 4.3  seconds.
---------------------------------------------


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [11:19<00:04,  5.00s/it]

The completed version is: The last major building work took place during the reign of King George V when , in 1913 , Sir Aston Webb redesigned Blore 's 1850 East Front to resemble a grand Norman church. After the First World War, Bl ore and St Michael 's, two of the finest medieval churches in Britain, were severely damaged by bombs and the new church and clo isters had to be pulled down and rebuilt during the 1920 s. A magnificent, three- story stained glass window was specially commissioned to replace the destroyed one in 1960- 61 which still stands on the West Front.
---------------------------------------------
The running time is 4.47  seconds.
---------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [11:24<00:00,  6.85s/it]

The completed version is: The departures of Núñez and van Gaal were hardly noticed by the fans when compared to that of Luís Figo , then club vice-captain . Figo had become a cult figure during the 1998 World Cup in France, in which Portugal lifted the trophy, and was expected to be at the heart of the club 's strategy in the coming season. But, as with a good deal of Portugal' a post- t ren era, it turned out a different story. Fig o had been involved in an altercation during a training session that night at the Est á p ode, and a broken leg had hampered him throughout the season.
---------------------------------------------
The running time is 5.25  seconds.
---------------------------------------------


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# save as json file

df = pd.read_csv('datasets/squad-4-one-token-fast-gpu.csv')
df_temp = df[['original', 'one-token']]
df_temp = df_temp.rename(columns={'one-token':'sampled'})

dict_temp = {'original':[], 'sampled':[]}
df_temp

In [62]:
for i in range(len(df_temp)):
    dict_temp['original'].append(df_temp.loc[i,'original'])
    dict_temp['sampled'].append(df_temp.loc[i,'sampled'])
    
out_file = open('datasets/squad_4-one-token-fast-gpu.raw_data.json', 'w') 
json.dump(dict_temp, out_file, indent = 4)
out_file.close() 

In [16]:
# !conda env export > example_saving.yml